<a href="https://colab.research.google.com/github/FiafRex/Cart-Pole-game/blob/main/Mainfile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Dependencies

In [ ]:
import random
import gym #by openAI to use with reinfornced DL
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

These lines import necessary libraries and modules for the code to run. These include libraries for handling randomness, interacting with the OpenAI Gym environment, working with arrays, using deque for experience replay, and building and training neural networks with Keras.

Start the environment

In [21]:
env = gym.make('CartPole-v1')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


This line creates an environment called 'CartPole-v1' using the Gym library. This environment represents a cart and pole balancing problem, where the goal is to balance a pole on a moving cart for as long as possible.

In [ ]:
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01

parameters related to exploration. Epsilon is the exploration rate, starting at 1.0 and decaying over time (at 0.995 rate).

In [22]:
def play_action(model, state):
    if np.random.rand() <= epsilon:
        return random.randrange(number_of_actions)
    else:
        action_values = model.predict(state)
        return np.argmax(action_values[0])


Implements agent's policy for selection actions. Taking the current state and the model (neural network) as input. With epsilon, it chooses a random action, otherwise selecting teh action with the highest predicted Q-value (expected reward) using the model

In [ ]:
gamma = 0.95


Defines discount facotr (gamma)-which deterimines the importance of future rewards in teh agent's decision making process. A value of 0.95-means that future rewards are discounted by 5% at each step.

In [ ]:
def fit(model, memory, epsilon):
    batch = random.sample(memory, batch_size)
    for state, action, reward, next_state, done in batch:
        target = reward
        if not done:
            target = reward + gamma * np.amax(model.predict(next_state)[0])
        state_target = model.predict(state)
        state_target[0][action] = target
        model.fit(state, state_target, epochs=1, verbose=0)
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay
    return model, epsilon


Implements the experience replay mechanism and updates model based on past expereiences. It samples a batch of expereinces from memory-calculating the target Q value for each experience, updates teh Q value of the chosen action in the current state, and trains the model using the current state and updated target values. It also decays epsilon, when and if its above the minimum value.

In [ ]:
state_size = env.observation_space.shape[0]
number_of_actions = env.action_space.n


These lines extract the state size and number of actions from the environment. The state size-represents the number of observations in the environment and the number of actions represents the number of possible actions, the agent can take.

In [ ]:
model = Sequential()
model.add(Dense(24, input_dim=state_size, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(number_of_actions, activation='linear'))


Code defines a neural network model using Kears, a simple feedforward network with two hidden layyers, containing 24 neurons and an output layer with anumber of neurons = number of ations. The activation functions used are ReLu for the hidden layer and linear for Output layer/

In [ ]:
model.compile(loss='mse', optimizer='adam')


Line compiles the model, specifies the loss function and optimizer

In [ ]:
memory = deque(maxlen=2000)
number_of_episodes = 1000
batch_size = 32


Initialize a memroy buffer to store past exp, set the number of training episodes and specify the batch size for experience replay.

In [ ]:
for e in range(number_of_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time_step in range(500):
        env.render()
        action = play_action(model, state)
        next_state, reward, done, _ = env.step(action)
        if done:
            reward = -15
        else:
            reward = reward
        next_state = np.reshape(next_state, [1, state_size])
        memory.append((state, action, reward, next_state, done))
        state = next_state
        if done:
            print("Number of points:", time_step)
            break
        if len(memory) > batch_size:
            model, epsilon = fit(model, memory, epsilon)


Main training loop for agenmt. It iterates over a specified number of eps, where each ep represents a complete game. In each ep, the agent interacts with the environment, selects actions based on policy-observes consequences, updates memroy with experiences and trains model using experience replay.